## **1. Installing the Required Library**

In [ ]:

# Install required libraries
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets
!pip install librosa soundfile
!pip install accelerate
!pip install wandb
!pip install audiomentations
!pip install git+https://github.com/huggingface/transformers.git

# Import essential libraries
import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    SpeechT5Processor,
    SpeechT5ForSpeechToSpeech,
    SpeechT5HifiGan
)
from datasets import load_dataset, Audio
import matplotlib.pyplot as plt
import IPython.display as ipd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchaudio]2 [torchaudio]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 35.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.4/803.4 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [datasets]/19 [datasets]ers]ub]

[notice] A new release of pip is available: 25.1.1 -> 25.3


## **2. Data Loading and Preprocessing**

### Step 1: Load Multilingual Speech Dataset

In [ ]:
from huggingface_hub import login

login()


In [11]:
# Load Mozilla Common Voice dataset (example with Hindi and English)
from datasets import load_dataset

# Load dataset - start with 2 languages for proof of concept
dataset_en = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="train[:1000]")
dataset_hi = load_dataset("mozilla-foundation/common_voice_13_0", "hi", split="train[:1000]")

# Load CoVoST dataset for speech-to-speech translation pairs
covost_dataset = load_dataset("covost2", "en_hi", split="train[:500]")

print(f"English samples: {len(dataset_en)}")
print(f"Hindi samples: {len(dataset_hi)}")
print(f"CoVoST pairs: {len(covost_dataset)}")


Repo card metadata block was not found. Setting CardData to empty.


EmptyDatasetError: The directory at hf://datasets/mozilla-foundation/common_voice_13_0@ff2bbb54dcdb597100fe534a1b911ff9103f9e22 doesn't contain any data files